## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-12-09-49-57 +0000,bbc,Madagascar presidency says attempt to seize po...,https://www.bbc.com/news/articles/cd070pn4g07o...
1,2025-10-12-09-31-53 +0000,bbc,Afghan Taliban says 58 Pakistani troops killed...,https://www.bbc.com/news/articles/cgmzmn4971zo...
2,2025-10-12-09-30-00 +0000,wsj,"To Find Workers, Hospitals Are Training Teenagers",https://www.wsj.com/us-news/education/healthca...
3,2025-10-12-09-30-00 +0000,wsj,Hyundai Factory Was a Deadly Job Site Before I...,https://www.wsj.com/us-news/hyundai-factory-wa...
4,2025-10-12-09-30-00 +0000,wsj,The Former Navy SEAL Who Built a Powerhouse Po...,https://www.wsj.com/business/media/who-is-shaw...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
93,trump,30
59,gaza,16
66,shutdown,12
215,tennessee,9
129,deal,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
142,2025-10-11-17-17-44 +0000,nypost,Putin chides Nobel winners who ‘did nothing fo...,https://nypost.com/2025/10/11/world-news/putin...,82
136,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...,81
115,2025-10-11-20-36-00 +0000,wsj,Trump said funds have been identified to pay m...,https://www.wsj.com/politics/policy/trump-says...,81
39,2025-10-12-04-37-28 +0000,bbc,Israelis praise Trump at huge rally ahead of e...,https://www.bbc.com/news/articles/c75q5qng0rgo...,76
179,2025-10-11-12-30-00 +0000,nypost,First hostage freed from Hamas’ grasp thanks T...,https://nypost.com/2025/10/11/world-news/first...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
142,82,2025-10-11-17-17-44 +0000,nypost,Putin chides Nobel winners who ‘did nothing fo...,https://nypost.com/2025/10/11/world-news/putin...
136,51,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...
139,41,2025-10-11-17-33-00 +0000,wsj,Authorities Find No Survivors After Deadly Bla...,https://www.wsj.com/us-news/tennessee-explosio...
53,38,2025-10-12-02-45-06 +0000,latimes,Former schools chief Austin Beutner plans to c...,https://www.latimes.com/california/story/2025-...
28,36,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...
103,29,2025-10-11-21-29-39 +0000,startribune,St. Paul police: 22-year-old man was assaulted...,https://www.startribune.com/st-paul-police-man...
89,29,2025-10-11-22-45-15 +0000,nypost,"Argentinian model, singer Fede Dorcaz shot and...",https://nypost.com/2025/10/11/world-news/argen...
169,29,2025-10-11-13-44-51 +0000,nypost,Joe Biden undergoing radiation in new phase of...,https://nypost.com/2025/10/11/us-news/joe-bide...
39,28,2025-10-12-04-37-28 +0000,bbc,Israelis praise Trump at huge rally ahead of e...,https://www.bbc.com/news/articles/c75q5qng0rgo...
6,25,2025-10-12-09-25-55 +0000,nypost,Emergency flights diverted from Portland hospi...,https://nypost.com/2025/10/12/us-news/portland...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
